In [ ]:
import numpy as np
from utils import *
import math
import matplotlib.pyplot as plt

In [ ]:
n=160 #choose 160 for a challenging case
d=3
def spiral_xy(i, spiral_num):
    """
    Create the data for a spiral.

    Arguments:
        i runs from 0 to 96
        spiral_num is 1 or -1
    """
    φ = i/16 * math.pi
    r = 6.5 * ((104 - i)/104)
    x = (r * math.cos(φ) * spiral_num)/13 + 0.5
    y = (r * math.sin(φ) * spiral_num)/13 + 0.5
    return (x, y)

def spiral(spiral_num):
    return [spiral_xy(i, spiral_num) for i in range(n//2)]

a = spiral(1)
b = spiral(-1)
X_raw=2*np.concatenate((a,b),axis=0)-1
y_raw=np.concatenate((np.ones(n//2),-np.ones(n//2)))

index = np.arange(n)
np.random.seed(1)
np.random.shuffle(index)
train_num = n
training_data_np = X_raw[index[:train_num],:]
training_labels_np = y_raw[index[:train_num]]
test_data_np = X_raw[index[train_num:],:]
test_labels_np = y_raw[index[train_num:]]

In [ ]:
# convex training
Hidden = 200
reg_p = 2
beta = 1e-3
params_dict = {}
method_list = ['Geometric_Algebra','Gaussian']
activation_list = ['ReLU']#,'gReLU']
np.random.seed(0)
sdim_list = [-1,1]
activation = 'ReLU'
for method in method_list:
#     for activation in activation_list:
    key = '{}'.format(method)
    params = cvx_solver_mosek(training_data_np,training_labels_np,arr_select=method,
                              Hidden=Hidden,sdim=-1,beta=beta,activation=activation)
    train_acc = cvx_solver_evaluate(training_data_np,training_labels_np,params,activation=activation)
    print('{} train: {:.2f}'.format(key, train_acc))
    params_dict[key] = params

In [ ]:
# convex lasso
reg_p = 2
method = 'GA_enum'#'Geometric_Algebra'#'Gaussian'#'Geometric_Algebra'
beta = 1e-3
activation = 'Lasso'
params_lasso = cvx_solver_mosek(training_data_np,training_labels_np,arr_select=method,sdim=-1,beta=beta,
                          activation=activation,verbose=True)
train_acc = cvx_solver_evaluate(training_data_np,training_labels_np,params_lasso,activation=activation)
print('train: {:.2f}'.format(train_acc))

In [ ]:
#subsampled lasso
reg_p = 2
method = 'Geometric_Algebra'#'Geometric_Algebra'#'Gaussian'#'Geometric_Algebra'
beta = 1e-3
activation = 'Lasso'
params_lasso_sub = cvx_solver_mosek(training_data_np,training_labels_np,arr_select=method,
                                    Hidden=Hidden,sdim=-1,beta=beta,
                          activation=activation,verbose=True,aug_sym=True)
train_acc = cvx_solver_evaluate(training_data_np,training_labels_np,params_lasso_sub,activation=activation)
print('train: {:.2f}'.format(train_acc))

In [ ]:
# nonconvex training
lr_list = [1e-1,1e-2,1e-3]
NN_dict = {}
train_acc_noncvx = -1
best_lr = -1
Epochs = 1000
set_seed(0)
for lr in lr_list:
    key = '{:.0e}'.format(lr)
    train_data = TensorDataset(torch.tensor(training_data_np).float(), torch.tensor(training_labels_np).float())
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=n, drop_last=True)
    NNclassifier, optimizer, scheduler = initialize_model(Hidden, D_in = 2, epochs=Epochs, lr=lr, beta=1e-3,train_dataloader=train_dataloader)
    cum_time, train_loss, test_loss, train_acc, test_acc = train(NNclassifier, optimizer, scheduler, train_dataloader, train_dataloader, epochs=Epochs, evaluation=True, freq_batch=1)
    NN_dict[key] = NNclassifier
    print('train acc: {:.2f}'.format(train_acc[-1]))
    if train_acc[-1]>train_acc_noncvx:
        best_lr = lr
        train_acc_noncvx = train_acc[-1]

In [ ]:
samp=200
x1=np.linspace(-1,1,samp).reshape(-1,1)
x2=np.linspace(-1,1,samp).reshape(-1,1)

Xtest=np.meshgrid(x1,x2)
# add the bias term
Xtest_raw = np.concatenate([Xtest[0].reshape(samp**2,1),Xtest[1].reshape(samp**2,1)],axis=1)
Xtest = np.concatenate([Xtest_raw,np.ones(samp**2).reshape(samp**2,1)],axis=1)

In [ ]:
best_key = '{:.0e}'.format(best_lr)
best_NN_classifier = NN_dict[best_key]
Xtest_t = torch.tensor(Xtest_raw).float()
yest_noncvx = torch.sign(best_NN_classifier(Xtest_t)).detach().numpy()
pos_noncvx=np.where(np.sign(yest_noncvx)==1)
neg_noncvx=np.where(np.sign(yest_noncvx)==-1)

In [ ]:
params_dict.keys()

In [ ]:
ind_dict = {}
X = X_raw.copy()
y = y_raw.copy()
for method in method_list:
    key = '{}'.format(method)
    params = params_dict[key]
    G, U = params
    activation = 'ReLU'
    if activation=='ReLU':
        yest_cvx=np.sum(relu(Xtest@G)-relu(Xtest@U),axis=1)
    elif activation=='gReLU':
        yest_cvx=np.sum(drelu(Xtest@G)*(Xtest@U),axis=1)

    pos_cvx=np.where(np.sign(yest_cvx)==1)
    neg_cvx=np.where(np.sign(yest_cvx)==-1)
    ind_dict[key] = [pos_cvx,neg_cvx]
        
pos = np.where(y==1)
neg = np.where(y==-1)
fig, axes = plt.subplots(2, 3,figsize=(12,8))

ax = axes[0,0]
ax.scatter(X[pos,0],X[pos,1],10,marker='>',c='y');
ax.scatter(X[neg,0],X[neg,1],10,marker='<',c='b');
ax.set_title("Training data")

ax = axes[0,1]

ax.scatter(Xtest[pos_noncvx,0],Xtest[pos_noncvx,1],2,marker='o',c='r');
ax.scatter(Xtest[neg_noncvx,0],Xtest[neg_noncvx,1],2,marker='o',c='g');
ax.scatter(X[pos,0],X[pos,1],10,marker='>',c='y');
ax.scatter(X[neg,0],X[neg,1],10,marker='<',c='b');
ax.set_title("Nonconvex AdamW")

ax = axes[0,2]
G, z, t = params_lasso
yest_cvx = relu(Xtest@G)@z+t
pos_cvx=np.where(np.sign(yest_cvx)==1)
neg_cvx=np.where(np.sign(yest_cvx)==-1)
ax.scatter(Xtest[pos_cvx,0],Xtest[pos_cvx,1],2,marker='o',c='r');
ax.scatter(Xtest[neg_cvx,0],Xtest[neg_cvx,1],2,marker='o',c='g');
ax.scatter(X[pos,0],X[pos,1],10,marker='>',c='y');
ax.scatter(X[neg,0],X[neg,1],10,marker='<',c='b');
ax.set_title("Convex {}".format('Lasso'))
ax = axes[1,2]
G, z, t = params_lasso_sub
yest_cvx = relu(Xtest@G)@z+t
pos_cvx=np.where(np.sign(yest_cvx)==1)
neg_cvx=np.where(np.sign(yest_cvx)==-1)
ax.scatter(Xtest[pos_cvx,0],Xtest[pos_cvx,1],2,marker='o',c='r');
ax.scatter(Xtest[neg_cvx,0],Xtest[neg_cvx,1],2,marker='o',c='g');
ax.scatter(X[pos,0],X[pos,1],10,marker='>',c='y');
ax.scatter(X[neg,0],X[neg,1],10,marker='<',c='b');
title = "Convex Lasso subsampled"
ax.set_title(title)

for i, method in enumerate(method_list):
    key = '{}'.format(method)
    pos_cvx,neg_cvx = ind_dict[key]
    ax = axes[1,i]
    ax.scatter(Xtest[pos_cvx,0],Xtest[pos_cvx,1],2,marker='o',c='r');
    ax.scatter(Xtest[neg_cvx,0],Xtest[neg_cvx,1],2,marker='o',c='g');
    ax.scatter(X[pos,0],X[pos,1],10,marker='>',c='y');
    ax.scatter(X[neg,0],X[neg,1],10,marker='<',c='b');
    title = "Convex {} {}".format(activation, method)
    ax.set_title(title)
fig.savefig('figures/Illus_spiral.pdf',bbox_inches='tight')

In [ ]:
# Lasso reg path based on cvxpy
p = 2
n, Embedding_Size = np.shape(training_data_np)
assert Embedding_Size==2, 'wrong input dimension'
G = np.zeros([3,n*(n-1)])
count = 0
for i in range(n):
    for j in range(i+1,n):
        xi = training_data_np[i]
        xj = training_data_np[j]
        v = (xi-xj)/np.linalg.norm(xi-xj)
        # rotate 90 degrees
        v = np.array([v[1],-v[0]])
        G[:2,count] = v
        G[2,count] = -xj@v
        G[:2,count+1] = -v
        G[2,count+1] = xi@v
        count+=2
training_data_aug = np.concatenate([training_data_np,np.ones([n,1])],axis=1)
Embedding_Size += 1

dmat= drelu(np.matmul(training_data_aug,G))

G = G/np.linalg.norm(G[:-1,:],p,axis=0)
X = relu(training_data_aug@G)
y = training_labels_np.copy()

beta_max = np.max(X.T@y)
n_betas = 10
beta_list = beta_max * np.geomspace(1, 1e-3, n_betas)
m1=G.shape[1]
z =cp.Variable(m1)
t = cp.Variable(1)
yopt = X@z+t
regularization = cp.norm(z,1)

params_list = []
for e, beta in tqdm(enumerate(beta_list)):
    cost=cp.sum_squares(yopt-training_labels_np)+beta*regularization
    prob=cp.Problem(cp.Minimize(cost))
    cvx_solver = cp.MOSEK
    prob.solve(solver=cvx_solver,verbose=False)
    z_v, t_v = z._value, t._value
    params_list.append([z_v, t_v])

In [ ]:
samp=200
x1=np.linspace(-1,1,samp).reshape(-1,1)
x2=np.linspace(-1,1,samp).reshape(-1,1)

Xtest=np.meshgrid(x1,x2)
# add the bias term
Xtest_raw = np.concatenate([Xtest[0].reshape(samp**2,1),Xtest[1].reshape(samp**2,1)],axis=1)
Xtest = np.concatenate([Xtest_raw,np.ones(samp**2).reshape(samp**2,1)],axis=1)
pos = np.where(y_raw==1)
neg = np.where(y_raw==-1)

In [ ]:
for e, beta in tqdm(enumerate(beta_list)):
    fig, ax = plt.subplots(1, 1,figsize=(8,8))
    z, t = params_list[e]
    yest_cvx = relu(Xtest@G)@z+t
    pos_cvx=np.where(np.sign(yest_cvx)==1)
    neg_cvx=np.where(np.sign(yest_cvx)==-1)
    ax.scatter(Xtest[pos_cvx,0],Xtest[pos_cvx,1],2,marker='o',c='r');
    ax.scatter(Xtest[neg_cvx,0],Xtest[neg_cvx,1],2,marker='o',c='g');
    ax.scatter(X_raw[pos,0],X_raw[pos,1],10,marker='>',c='y');
    ax.scatter(X_raw[neg,0],X_raw[neg,1],10,marker='<',c='b');
    title = r"Convex Lasso $\beta$={:.2e}".format(beta)
    ax.set_title(title)
    fig.savefig('figures/lasso_path{}.png'.format(e))